In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
df=pd.read_csv(r"C:\Users\Admin\Documents\SEM 5 CLG\AI\FINAL_DATA.csv", sep=',', encoding='windows-1252')

In [6]:
df.head(10)

,worker name,job title,skills,rating,location
0,Sonnnie Belham,electrician,solve all electrical issues,4,3 East Parkway
1,Brendon Axleby,accountant,accounting,4,1772 Orin Point
2,Quentin Calcut,painter,"painting houses,buildings",1,217 Arapahoe Lane
3,Glenine Drayton,shop worker,working in various shops,0,57224 Maryland Terrace
4,Neel Lodewick,washing machine repairing,repair all types of washing machines,0,43 Scoville Road
5,Roderich Desouza,carpenter,furniture making,0,5192 Cambridge Crossing
6,Bunnie McElvogue,cleaning worker,cleaning,0,8 West Crossing
7,Warren Light,baker,"cook cakes,creativity,skilled in using ovens",1,9 Emmet Crossing
8,Odetta How to preserve,shop worker,working in various shops,2,6628 Ohio Terrace
9,Siffre Chitter,care taker,taking care of old people and small children,3,25 Pankratz Place


In [7]:
df['job title']=df['job title'].str.strip()
df['job title']=df['job title'].str.lower()
df['skills']=df['skills'].str.strip()
df['skills']=df['skills'].str.lower()


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv=TfidfVectorizer(min_df=3, max_features=None,
                   strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}\S',
                   ngram_range=(1,3),
                   stop_words='english')
df['job title']=df['job title'].fillna('')
df['skills']=df['skills'].fillna('')
df['Job']=df['job title']+" "+df['skills']

In [9]:
tfv_matrix=tfv.fit_transform(df['Job'])

In [10]:
tfv_matrix

<500x275 sparse matrix of type '<class 'numpy.float64'>'
	with 4780 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig=sigmoid_kernel(tfv_matrix,tfv_matrix)

In [12]:
sig[0]

array([0.76311711, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76311711, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76169088, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76311711, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76169088, 0.76159416, 0.76311711, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76169088, 0.76311711, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159

In [13]:
ind=pd.Series(df['job title']).drop_duplicates(keep='first')
print(ind.index)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   9,  12,  14,  16,  19,
             21,  29,  31,  32,  33,  35,  50,  51,  63,  76,  90,  94, 114,
            119, 124, 130, 148, 151, 157, 162, 166, 181, 211, 212, 221, 227,
            245, 246, 264, 276, 285, 298, 304, 345, 384, 399, 407, 417],
           dtype='int64')


In [14]:
index_=ind
indices=pd.Series(ind.index)
indices.index=index_

In [15]:
indices

job title
electrician                    0
accountant                     1
painter                        2
shop worker                    3
washing machine repairing      4
carpenter                      5
cleaning worker                6
baker                          7
care taker                     9
web developer                 12
app developer                 14
architect                     16
tester                        19
plumber                       21
bartender                     29
social worker                 31
mobile repairing              32
software architect            33
sweeper                       35
barber                        50
masseur                       51
automobile repairing          63
broiler cook                  76
tv repairing                  90
catering manager              94
washing machine              114
farmworker                   119
beautician                   124
cooking                      130
executive cook               148


In [16]:
def give_rec(title , sig=sig):
    try:
        idx=indices[title]
        sig_scores=list(enumerate(sig[idx]))
        sig_scores=sorted(sig_scores, key=lambda x: x[1],reverse=True)
        sig_scores=sig_scores[1:8]
        customer_indices =[i[0] for i in sig_scores]
        values=df.iloc[customer_indices]
        #values.sort_values(by = 'Experience' ,ascending=False)
       # return df.iloc[customer_indices]
        return values.sort_values(by = 'rating' ,ascending=False)
    except:
        contain_values=df[df['Job'].str.contains(title)]
        id=contain_values.index
        df.loc[id]
        print(len(id))
        if len(id)<10:
            for j in range(0,len(id)): 
                if j==0:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores2[1:5]
                else:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores+sig_scores2[1:3]
        else:
            for j in range(4,9): 
                if j==4:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores2[1:5]
                else:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores+sig_scores2[1:2]
        customer_indices=[i[0] for i in sig_scores]
        value=df.iloc[customer_indices]
        print(value.sort_values(by = 'rating' ,ascending=False))

In [17]:
give_rec('repairing')

6
             worker name                  job title  \
202          Leah Shayes            washing machine   
162           Thea Loidl           cooler repairing   
162           Thea Loidl           cooler repairing   
114       Barret Kiessel            washing machine   
63      Krystalle Kaplin       automobile repairing   
63      Krystalle Kaplin       automobile repairing   
63      Krystalle Kaplin       automobile repairing   
184  Amelie Bickerstasse            washing machine   
90            Dolf Diack               tv repairing   
90            Dolf Diack               tv repairing   
135      Janene Bernette            washing machine   
151     Loralee Saltrese           fridge repairing   
4          Neel Lodewick  washing machine repairing   
32       Carly Edlington           mobile repairing   

                                          skills  rating  \
202                all types of washing machines       5   
162     motor exchange,cooling pad repairing etc    

In [18]:
give_rec('driver')

8
            worker name     job title  \
264          Elysee Fri   taxi driver   
278  Felisha Nan Carrow   taxi driver   
278  Felisha Nan Carrow   taxi driver   
278  Felisha Nan Carrow   taxi driver   
318       Carissa Stubs  truck driver   
318       Carissa Stubs  truck driver   
318       Carissa Stubs  truck driver   
318       Carissa Stubs  truck driver   
318       Carissa Stubs  truck driver   
274   Joaquin Galbraeth  truck driver   
274   Joaquin Galbraeth  truck driver   
467      Veradis Essery  truck driver   
338     Joshuah Hansill  truck driver   
338     Joshuah Hansill  truck driver   
338     Joshuah Hansill  truck driver   
338     Joshuah Hansill  truck driver   
360        Arvy Crowden  truck driver   
338     Joshuah Hansill  truck driver   

                                          skills  rating  \
264              can drive any type of 4 whiller       5   
278                             for one day trip       4   
278                             for on

In [19]:
give_rec('cook')

21
               worker name            job title  \
275         Filippa Dengel       fry/saute cook   
492       Kimmy Sherington      kitchen manager   
263          Giffard Bolin          pantry cook   
263          Giffard Bolin          pantry cook   
227             Ripley Nix  soup and sauce cook   
211  Virgilio Bartholomaus          pantry cook   
384          Ravi Guiducci            line cook   
29         Antonie Duckham            bartender   

                                           skills  rating  \
275                 expert in fried sauteed items       4   
492                       handles cooking/kitchen       4   
263                               cold food items       3   
263                               cold food items       3   
227                           all soups and sauce       2   
211                               cold food items       1   
384  responsible for daily food and kitchen tasks       1   
29                          serves the drink,food

In [40]:
give_rec('cleaning')

46
          worker name        job title                          skills  \
80      Elna Corragan          sweeper  cleaning and maintaining house   
99   Jeremias McFaell          sweeper  cleaning and maintaining house   
49      Shanda Neward          sweeper  cleaning and maintaining house   
49      Shanda Neward          sweeper  cleaning and maintaining house   
49      Shanda Neward          sweeper  cleaning and maintaining house   
64       Ediva Bridle          sweeper  cleaning and maintaining house   
20  Leonidas Bausmann  cleaning worker                        cleaning   
20  Leonidas Bausmann  cleaning worker                        cleaning   

    rating               location                                     Job  
80       5       110 Elmside Pass  sweeper cleaning and maintaining house  
99       5      5 New Castle Lane  sweeper cleaning and maintaining house  
49       3     3 Barnett Junction  sweeper cleaning and maintaining house  
49       3     3 Barnett J

In [41]:
 give_rec('furniture making')

24
          worker name  job title            skills  rating  \
13   Carolee Rembrant  carpenter  furniture making       5   
34  Archibold Vasiliu  carpenter  furniture making       5   
55    Malissa Satchel  carpenter  furniture making       5   
13   Carolee Rembrant  carpenter  furniture making       5   
13   Carolee Rembrant  carpenter  furniture making       5   
13   Carolee Rembrant  carpenter  furniture making       5   
13   Carolee Rembrant  carpenter  furniture making       5   
22         Willi Bury  carpenter  furniture making       0   

                 location                         Job  
13        0097 Towne Road  carpenter furniture making  
34       56 Warbler Plaza  carpenter furniture making  
55  72178 John Wall Alley  carpenter furniture making  
13        0097 Towne Road  carpenter furniture making  
13        0097 Towne Road  carpenter furniture making  
13        0097 Towne Road  carpenter furniture making  
13        0097 Towne Road  carpenter furniture

In [46]:
give_rec('cook')

21
               worker name            job title  \
275         Filippa Dengel       fry/saute cook   
492       Kimmy Sherington      kitchen manager   
263          Giffard Bolin          pantry cook   
263          Giffard Bolin          pantry cook   
227             Ripley Nix  soup and sauce cook   
211  Virgilio Bartholomaus          pantry cook   
384          Ravi Guiducci            line cook   
29         Antonie Duckham            bartender   

                                           skills  rating  \
275                 expert in fried sauteed items       4   
492                       handles cooking/kitchen       4   
263                               cold food items       3   
263                               cold food items       3   
227                           all soups and sauce       2   
211                               cold food items       1   
384  responsible for daily food and kitchen tasks       1   
29                          serves the drink,food